In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import heapq
from collections import deque
import time

In [4]:
# ================== مرحله 1: بارگذاری داده‌ها ==================
distances_df = pd.read_excel("/content/drive/MyDrive/ProvinceCenterDistances.xlsx")
neighbours_df = pd.read_excel("/content/drive/MyDrive/ProvinceCentersNeighbours.xlsx")
straight_line_df = pd.read_excel("/content/drive/MyDrive/ProvinceCentersStraightLineDistances.xlsx")

print("داده‌ها با موفقیت بارگذاری شدند")
print(f"تعداد شهرها: {len(distances_df)}")

داده‌ها با موفقیت بارگذاری شدند
تعداد شهرها: 31


In [5]:
# ================== مرحله 2: ساخت گراف ==================
def build_graph(distances_df, neighbours_df):
    """
    ساخت گراف از داده‌های فاصله و همسایگی
    گراف به صورت دیکشنری از دیکشنری‌ها: {city: {neighbor: distance}}
    """
    graph = {}

    # لیست شهرها
    cities = distances_df['Unnamed: 0'].tolist()

    # مقداردهی اولیه گراف
    for city in cities:
        graph[city] = {}

    # اضافه کردن یال‌ها بر اساس فایل همسایگی
    for idx, row in neighbours_df.iterrows():
        city1 = row['Unnamed: 0']
        for city2 in neighbours_df.columns[1:]:
            if pd.notna(row[city2]) and row[city2] != 0:
                # پیدا کردن فاصله از جدول فاصله‌ها
                city1_idx = distances_df[distances_df['Unnamed: 0'] == city1].index[0]
                distance = distances_df.loc[city1_idx, city2]
                if distance > 0:
                    graph[city1][city2] = distance

    return graph, cities

graph, cities = build_graph(distances_df, neighbours_df)
print(f"\nگراف ساخته شد با {len(graph)} گره")
print(f"نمونه همسایگان تهران: {list(graph['تهران'].keys())[:5]}")


گراف ساخته شد با 31 گره
نمونه همسایگان تهران: ['اراک', 'ساری', 'سمنان', 'قزوین', 'قم']


In [6]:
# ================== مرحله 3: پیاده‌سازی الگوریتم‌ها ==================

# ------------------ BFS ------------------
def bfs(graph, start, goal):
    """
    الگوریتم جستجوی سطح-اول (BFS)
    برمی‌گرداند: (مسیر, تعداد گره‌های بازدیدی, طول مسیر)
    """
    if start not in graph or goal not in graph:
        return None, 0, 0

    if start == goal:
        return [start], 1, 0

    # صف برای BFS
    queue = deque([(start, [start], 0)])  # (گره فعلی, مسیر, طول مسیر)
    visited = {start}
    nodes_visited = 0

    while queue:
        current, path, path_length = queue.popleft()
        nodes_visited += 1

        # بررسی همسایگان
        for neighbor, distance in graph[current].items():
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = path + [neighbor]
                new_length = path_length + distance

                if neighbor == goal:
                    return new_path, nodes_visited, new_length

                queue.append((neighbor, new_path, new_length))

    return None, nodes_visited, 0


# ------------------ DFS ------------------
def dfs(graph, start, goal):
    """
    الگوریتم جستجوی عمق-اول (DFS)
    برمی‌گرداند: (مسیر, تعداد گره‌های بازدیدی, طول مسیر)
    """
    if start not in graph or goal not in graph:
        return None, 0, 0

    if start == goal:
        return [start], 1, 0

    # پشته برای DFS
    stack = [(start, [start], 0)]  # (گره فعلی, مسیر, طول مسیر)
    visited = set()
    nodes_visited = 0

    while stack:
        current, path, path_length = stack.pop()

        if current in visited:
            continue

        visited.add(current)
        nodes_visited += 1

        if current == goal:
            return path, nodes_visited, path_length

        # اضافه کردن همسایگان به پشته
        for neighbor, distance in graph[current].items():
            if neighbor not in visited:
                new_path = path + [neighbor]
                new_length = path_length + distance
                stack.append((neighbor, new_path, new_length))

    return None, nodes_visited, 0


# ------------------ Heuristic Function ------------------
def heuristic(city1, city2, df):
    """
    تابع heuristic بر اساس فاصله خط مستقیم بین دو شهر
    این heuristic هم admissible و هم consistent است
    """
    try:
        city1_idx = df[df['Unnamed: 0'] == city1].index[0]
        h_value = df.loc[city1_idx, city2]
        return h_value
    except:
        return 0


# ------------------ A* ------------------
def a_star(graph, start, goal, heuristic_func):
    """
    الگوریتم A* با استفاده از heuristic
    برمی‌گرداند: (مسیر, تعداد گره‌های بازدیدی, طول مسیر)
    """
    if start not in graph or goal not in graph:
        return None, 0, 0

    if start == goal:
        return [start], 1, 0

    # صف اولویت‌دار: (f(n), g(n), گره, مسیر)
    open_set = [(0, 0, start, [start])]
    visited = set()
    g_scores = {start: 0}
    nodes_visited = 0

    while open_set:
        f, g, current, path = heapq.heappop(open_set)

        if current in visited:
            continue

        visited.add(current)
        nodes_visited += 1

        if current == goal:
            return path, nodes_visited, g

        # بررسی همسایگان
        for neighbor, distance in graph[current].items():
            if neighbor not in visited:
                new_g = g + distance

                # اگر مسیر بهتری پیدا کردیم
                if neighbor not in g_scores or new_g < g_scores[neighbor]:
                    g_scores[neighbor] = new_g
                    h = heuristic_func(neighbor, goal, straight_line_df)
                    f = new_g + h
                    new_path = path + [neighbor]
                    heapq.heappush(open_set, (f, new_g, neighbor, new_path))

    return None, nodes_visited, 0


# ------------------ Weighted A* ------------------
def weighted_a_star(graph, start, goal, heuristic_func, w):
    """
    الگوریتم Weighted A* با ضریب وزن w
    f(n) = g(n) + w * h(n)
    برمی‌گرداند: (مسیر, تعداد گره‌های بازدیدی, طول مسیر)
    """
    if start not in graph or goal not in graph:
        return None, 0, 0

    if start == goal:
        return [start], 1, 0

    # صف اولویت‌دار: (f(n), g(n), گره, مسیر)
    open_set = [(0, 0, start, [start])]
    visited = set()
    g_scores = {start: 0}
    nodes_visited = 0

    while open_set:
        f, g, current, path = heapq.heappop(open_set)

        if current in visited:
            continue

        visited.add(current)
        nodes_visited += 1

        if current == goal:
            return path, nodes_visited, g

        # بررسی همسایگان
        for neighbor, distance in graph[current].items():
            if neighbor not in visited:
                new_g = g + distance

                # اگر مسیر بهتری پیدا کردیم
                if neighbor not in g_scores or new_g < g_scores[neighbor]:
                    g_scores[neighbor] = new_g
                    h = heuristic_func(neighbor, goal, straight_line_df)
                    f = new_g + (w * h)  # تفاوت اصلی: ضرب h در w
                    new_path = path + [neighbor]
                    heapq.heappush(open_set, (f, new_g, neighbor, new_path))

    return None, nodes_visited, 0

In [9]:
# ================== تست و مقایسه الگوریتم‌ها ==================

def test_algorithm(name, algorithm, graph, start, goal, **kwargs):
    """
    تست یک الگوریتم و اندازه‌گیری زمان و نتایج
    """
    start_time = time.time()
    path, nodes_visited, path_length = algorithm(graph, start, goal, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time

    return {
        'الگوریتم': name,
        'مسیر': ' → '.join(path) if path else 'یافت نشد',
        'طول مسیر (km)': path_length if path else 0,
        'تعداد گره‌های بازدیدی': nodes_visited,
        'زمان اجرا (s)': f"{execution_time:.6f}"
    }


def print_results_table(results):
    """
    چاپ جدول نتایج با فرمت زیبا
    """
    # محاسبه عرض ستون‌ها
    col_widths = {
        'الگوریتم': 20,
        'مسیر': 50,
        'طول مسیر (km)': 18,
        'تعداد گره‌های بازدیدی': 25,
        'زمان اجرا (s)': 15
    }

    # چاپ هدر
    print("\n" + "┌" + "─" * (sum(col_widths.values()) + len(col_widths) * 3 - 1) + "┐")

    header = "│"
    for col, width in col_widths.items():
        header += f" {col:^{width}} │"
    print(header)

    print("├" + "─" * (sum(col_widths.values()) + len(col_widths) * 3 - 1) + "┤")

    # چاپ ردیف‌ها
    for result in results:
        row = "│"
        row += f" {result['الگوریتم']:<{col_widths['الگوریتم']}} │"
        row += f" {result['مسیر']:<{col_widths['مسیر']}} │"
        row += f" {result['طول مسیر (km)']:>{col_widths['طول مسیر (km)']}} │"
        row += f" {result['تعداد گره‌های بازدیدی']:>{col_widths['تعداد گره‌های بازدیدی']}} │"
        row += f" {result['زمان اجرا (s)']:>{col_widths['زمان اجرا (s)']}} │"
        print(row)

    print("└" + "─" * (sum(col_widths.values()) + len(col_widths) * 3 - 1) + "┘")


# مسیرهای تست
test_routes = [
    ('تبریز', 'بندرعباس'),
    ('تهران', 'اصفهان')
]

for start_city, goal_city in test_routes:
    print(f"\n{'='*130}")
    print(f"{'مسیر: ' + start_city + ' → ' + goal_city:^130}")
    print('='*130)

    results = []

    # BFS
    print("\n⏳ در حال اجرای BFS...")
    results.append(test_algorithm('BFS', bfs, graph, start_city, goal_city))

    # DFS
    print("⏳ در حال اجرای DFS...")
    results.append(test_algorithm('DFS', dfs, graph, start_city, goal_city))

    # A*
    print("⏳ در حال اجرای A*...")
    results.append(test_algorithm('A*', a_star, graph, start_city, goal_city,
                                  heuristic_func=heuristic))

    # Weighted A* با مقادیر مختلف w
    for w in [1.5, 2.0, 3.0]:
        print(f"⏳ در حال اجرای Weighted A* (w={w})...")
        name = f'Weighted A* (w={w})'
        results.append(test_algorithm(name, weighted_a_star, graph,
                                     start_city, goal_city,
                                     heuristic_func=heuristic, w=w))

    # نمایش نتایج با جدول
    print_results_table(results)

    # یافتن بهترین نتایج
    best_length = min([r['طول مسیر (km)'] for r in results if r['طول مسیر (km)'] > 0])
    best_time = min([float(r['زمان اجرا (s)']) for r in results])
    best_nodes = min([r['تعداد گره‌های بازدیدی'] for r in results])

    print(f"\n🏆 بهترین طول مسیر: {best_length} km")
    print(f"⚡ سریع‌ترین زمان اجرا: {best_time:.6f} s")
    print(f"🔍 کمترین گره بازدیدی: {best_nodes}")


# ================== تحلیل و مقایسه ==================
print("\n" + "="*70)
print("تحلیل الگوریتم‌ها:")
print("="*70)

analysis = """
1. مقایسه Completeness و Optimality:
   - BFS: Complete ✓, Optimal در گراف بدون وزن ✓
   - DFS: Complete در گراف متناهی ✓, Optimal ✗
   - A*: Complete ✓, Optimal با heuristic admissible ✓
   - Weighted A*: Complete ✓, Optimal ✗ (برای w > 1)

2. Time & Space Complexity:
   - BFS: O(b^d) برای هر دو
   - DFS: O(b^m) برای زمان, O(bm) برای فضا
   - A*: O(b^d) در بدترین حالت
   - Weighted A*: بهتر از A* اما وابسته به w

3. مشکل DFS:
   - ممکن است در حلقه‌های بینهایت گیر کند
   - مسیر بهینه را پیدا نمی‌کند
   - اما: حافظه کمتری مصرف می‌کند و در برخی مسائل سریع‌تر است

4. Weighted A* vs A*:
   - سریع‌تر اما کمتر بهینه
   - مناسب برای مسائل real-time و بازی‌ها
   - trade-off بین سرعت و کیفیت

5. Admissibility و Consistency of Heuristic:
   - h(n) = فاصله خط مستقیم
   - Admissible ✓: h(n) ≤ h*(n) همیشه
   - Consistent ✓: h(n) ≤ c(n,n') + h(n')
   (فاصله خط مستقیم همیشه کمتر از مسیر واقعی است)
"""

print(analysis)


                                                      مسیر: تبریز → بندرعباس                                                      

⏳ در حال اجرای BFS...
⏳ در حال اجرای DFS...
⏳ در حال اجرای A*...
⏳ در حال اجرای Weighted A* (w=1.5)...
⏳ در حال اجرای Weighted A* (w=2.0)...
⏳ در حال اجرای Weighted A* (w=3.0)...

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│       الگوریتم       │                        مسیر                        │   طول مسیر (km)    │   تعداد گره‌های بازدیدی   │  زمان اجرا (s)  │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ BFS                  │ تبریز → زنجان → قزوین → کرمان → بندرعباس           │               2058 │                        14 │        0.000067 │
│ DFS                  │ تبریز → زنجان → همدان → کرمانشاه → قزوین → یزد → کرمان → شیراز → یاسوج → شهرکرد → 